# Reading the data

In [6]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [7]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("karpathy/tiny_shakespeare" ,trust_remote_code=True,split = 'train')

# View first few lines
print(dataset[0]['text'][:1000])  # print first 1000 characters

README.md:   0%|          | 0.00/6.10k [00:00<?, ?B/s]

tiny_shakespeare.py:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [8]:
dataset.shape

(1, 1)

# Data Preprocessing

### Word tokenization

In [9]:
 #WordPiece tokenization

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

text = dataset[0]['text']
tokens_WP = tokenizer.tokenize(text)
print(tokens_WP[:100])  # ['un', '##believable']

Token indices sequence length is longer than the specified maximum sequence length for this model (258333 > 512). Running this sequence through the model will result in indexing errors


['first', 'citizen', ':', 'before', 'we', 'proceed', 'any', 'further', ',', 'hear', 'me', 'speak', '.', 'all', ':', 'speak', ',', 'speak', '.', 'first', 'citizen', ':', 'you', 'are', 'all', 'resolved', 'rather', 'to', 'die', 'than', 'to', 'fa', '##mis', '##h', '?', 'all', ':', 'resolved', '.', 'resolved', '.', 'first', 'citizen', ':', 'first', ',', 'you', 'know', 'cai', '##us', 'marc', '##ius', 'is', 'chief', 'enemy', 'to', 'the', 'people', '.', 'all', ':', 'we', 'know', "'", 't', ',', 'we', 'know', "'", 't', '.', 'first', 'citizen', ':', 'let', 'us', 'kill', 'him', ',', 'and', 'we', "'", 'll', 'have', 'corn', 'at', 'our', 'own', 'price', '.', 'is', "'", 't', 'a', 'verdict', '?', 'all', ':', 'no', 'more']


In [10]:
#GPT-like model, BPE-style behavior
tokenizer = AutoTokenizer.from_pretrained("gpt2")

text = dataset[0]['text']
tokens_BPE = tokenizer.tokenize(text[:100])
print(tokens_BPE[:100])  # ['un', 'believable'] or ['unbel', 'ievable'] (depending on merges)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

['First', 'ĠCitizen', ':', 'Ċ', 'Before', 'Ġwe', 'Ġproceed', 'Ġany', 'Ġfurther', ',', 'Ġhear', 'Ġme', 'Ġspeak', '.', 'Ċ', 'Ċ', 'All', ':', 'Ċ', 'Spe', 'ak', ',', 'Ġspeak', '.', 'Ċ', 'Ċ', 'First', 'ĠCitizen', ':', 'Ċ', 'You']


BPE make spaces into special characters.We will continue with WordPiece

In [11]:
#Stemming + Lemmatizing
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk

# Make sure these are downloaded
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

df_tokens = pd.DataFrame({'Token': tokens_WP})

# Apply stemming and lemmatization
df_tokens['Stem'] = df_tokens['Token'].apply(lambda x: stemmer.stem(x))
df_tokens['Lemma'] = df_tokens['Token'].apply(lambda x: lemmatizer.lemmatize(x))

df_tokens


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


,Token,Stem,Lemma
0,first,first,first
1,citizen,citizen,citizen
2,:,:,:
3,before,befor,before
4,we,we,we
...,...,...,...
258328,?,?,?
258329,but,but,but
258330,who,who,who
258331,comes,come,come


In [12]:
#Show words that have different lemmatized and stemmed versions
df_tokens[df_tokens['Stem'] != df_tokens['Lemma']]

,Token,Stem,Lemma
3,before,befor,before
6,any,ani,any
25,resolved,resolv,resolved
32,##mis,##mi,##mis
37,resolved,resolv,resolved
...,...,...,...
258284,wedding,wed,wedding
258312,occasion,occas,occasion
258314,revenge,reveng,revenge
258322,thus,thu,thus


As we know, stem just truncates the words. It is faster but the stemmed chunks necessarily do not have meaning. Since we want to contnue the text in a meaningful way later, we will go with lemmatization.

Now we will do Lemmatizing based on part of speech. So, it understands that if ran was a verb it will convert it to run

In [13]:
import nltk

nltk.download('punkt_eng')                        # for tokenizing
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')   # for POS tagging
nltk.download('wordnet')                      # for lemmatization
nltk.download('omw-1.4')                      # for lemmatizer language mappings


[nltk_data] Error loading punkt_eng: Package 'punkt_eng' not found in
[nltk_data]     index
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [14]:
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default

lemmatizer = WordNetLemmatizer()

text = dataset[0]['text']
words = word_tokenize(text)
tagged = pos_tag(words)
lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged]

df_pos_lem = pd.DataFrame({'Words': words, 'Part of Speech': tagged,'Lemmatized': lemmatized_tokens})
df_pos_lem

,Words,Part of Speech,Lemmatized
0,First,"(First, RB)",First
1,Citizen,"(Citizen, NNP)",Citizen
2,:,"(:, :)",:
3,Before,"(Before, IN)",Before
4,we,"(we, PRP)",we
...,...,...,...
228773,?,"(?, .)",?
228774,But,"(But, CC)",But
228775,who,"(who, WP)",who
228776,comes,"(comes, VBZ)",come


In [15]:
#Check words that their memmatized is different
df_pos_lem[df_pos_lem['Lemmatized'] != df_pos_lem['Words']]

,Words,Part of Speech,Lemmatized
23,are,"(are, VBP)",be
25,resolved,"(resolved, VBD)",resolve
37,resolved,"(resolved, VBN)",resolve
48,is,"(is, VBZ)",be
67,us,"(us, PRP)",u
...,...,...,...
228716,is,"(is, VBZ)",be
228741,apes,"(apes, NNS)",ape
228770,grieved,"(grieved, VBN)",grieve
228771,as,"(as, IN)",a


### Noteabout differences between work tokens and phrases:

Token-level training learns small syntax rules, grammar, and short dependencies

Phrase-level training captures style, structure of speech, and broader semantic units

### Generate sequences of varying length phrases

In [16]:
#Create Phrase level sequences

import re

phrases = re.split(r'[,:;]', text)
phrases = [p.strip() for p in phrases if len(p.strip()) > 0]

# Lemmatize each phrase (with POS-aware)
def lemmatize_phrase(phrase):
    words = word_tokenize(phrase)
    tagged = pos_tag(words)
    return [lemmatizer.lemmatize(w, get_wordnet_pos(t)) for w, t in tagged]

phrase_data = [{'Type': 'Phrase', 'Raw': p, 'Lemmatized': lemmatize_phrase(p)} for p in phrases]

import pandas as pd
df_phrases = pd.DataFrame(phrase_data)
df_phrases.head()

,Type,Raw,Lemmatized
0,Phrase,First Citizen,"[First, Citizen]"
1,Phrase,Before we proceed any further,"[Before, we, proceed, any, further]"
2,Phrase,hear me speak.\n\nAll,"[hear, me, speak, ., All]"
3,Phrase,Speak,[Speak]
4,Phrase,speak.\n\nFirst Citizen,"[speak, ., First, Citizen]"


In [17]:
#Check when the Raw and Lemmatized version are different (Some words has changed)

# Clean the Raw column: remove \n, \t, and strip spaces
df_phrases['Raw_Cleaned'] = df_phrases['Raw'].apply(lambda x: ' '.join(x.split()))

import re

# Join tokens and fix spacing around punctuation
def clean_lemmatized(tokens):
    joined = ' '.join(tokens)
    # Remove space before punctuation (e.g., 'word .' → 'word.')
    cleaned = re.sub(r'\s+([.,!?;:])', r'\1', joined)
    return cleaned

df_phrases['Lemmatized_Joined'] = df_phrases['Lemmatized'].apply(clean_lemmatized)

# Compare cleaned raw text with the lemmatized version
changed = df_phrases[df_phrases['Raw_Cleaned'] != df_phrases['Lemmatized_Joined']]

# Show the differences
changed[['Raw_Cleaned', 'Lemmatized_Joined']].head()

,Raw_Cleaned,Lemmatized_Joined
5,You are all resolved rather to die than to fam...,You be all resolve rather to die than to famis...
6,Resolved. resolved. First Citizen,Resolved. resolve. First Citizen
8,you know Caius Marcius is chief enemy to the p...,you know Caius Marcius be chief enemy to the p...
11,Let us kill him,Let u kill him
12,and we'll have corn at our own price. Is't a v...,and we 'll have corn at our own price. Is't a ...


Based on this observation, I realized Lemmatizing is case snsitive

* Lowercase = verb or noun (contextualized)
* Capitalized = likely a proper noun or unknown, so it might not lemmatize it.

But on the other hand we are working with sentances data so cases are important so I decided not to make a change.

In [18]:
#Create Long phrases

from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(text)
chunks = [' '.join(sentences[i:i+5]) for i in range(0, len(sentences), 5)]

chunk_data = [{'Type': 'LongPhrase', 'Raw': chunk, 'Lemmatized': lemmatize_phrase(chunk)} for chunk in chunks]
df_long = pd.DataFrame(chunk_data)

print(df_long['Raw'][0])
print(' '.join(df_long['Lemmatized'][0]))

First Citizen:
Before we proceed any further, hear me speak. All:
Speak, speak. First Citizen:
You are all resolved rather to die than to famish? All:
Resolved. resolved.
First Citizen : Before we proceed any further , hear me speak . All : Speak , speak . First Citizen : You be all resolve rather to die than to famish ? All : Resolved . resolve .


# RNN architectures

**RNNs (Recurrent Neural Networks)** are the simplest form of sequence models. They process input sequentially, maintaining a hidden state that captures previous information. However, they struggle with long-range dependencies due to the vanishing gradient problem, which makes them less effective at capturing deeper context in text generation.

**LSTMs (Long Short-Term Memory networks)** are an improved version of RNNs. They include memory cells and gating mechanisms (input, output, forget gates) that allow them to retain information over longer sequences. This makes them much better suited for text generation and prediction, especially when understanding structure and long-term context is important.

**GRUs (Gated Recurrent Units)** are similar to LSTMs but have a simpler architecture with fewer gates (reset and update gates). They are computationally lighter and often perform comparably to LSTMs on many tasks. For text generation, GRUs strike a good balance between performance and training efficiency, especially when long sequences are involved but not excessively complex.

### Vanilla RNN (Bidirectional)

Vanilla RNN: Simple and fast for small tasks, good as a baseline.

Bidirectional: Captures both past and future context, especially helpful in NLP.

No dropout: We'll start clean; dropout will be added later

rnn_out[:, -1, :]: We use the last timestep output to feed into the classifier.

hidden_dim * 2: Due to bidirectionality (forward + backward).

In [27]:
import torch
import torch.nn as nn

class VanillaRNNModel_Bi(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(VanillaRNNModel_Bi, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)                      # (B, S, E)
        rnn_out, _ = self.rnn(embedded)                   # (B, S, H*2)
        return self.fc(rnn_out)                            # (B, S, V)


### LSTM (Bidirectional)

LSTM: Handles long-term dependencies better than RNN via gates (input, forget, output).

Bidirectional: Essential in many NLP tasks where context matters on both ends (e.g., sentiment).

Hidden size doubled for bidirectional=True.

Output at last timestep used — could also try mean/attention pooling later.

In [44]:
class LSTMModel_Bi(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers=1):
        super(LSTMModel_Bi, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # *2 for bidirectional

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        return self.fc(output)


### GRU (Bidirectional)

GRU: Simplified version of LSTM (fewer gates), faster to train, similar performance.

Often performs well on medium-sized datasets.

Same architecture as LSTM, just the recurrent unit changes.

In [50]:
class GRUModel_Bi(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(GRUModel_Bi, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)                      # (B, S, E)
        output, _ = self.gru(embedded)                   # (B, S, H*2)
        return self.fc(output)                            # (B, S, V)

# Train and Optimize Models

### Splitting data in text train validation

We decided to have training set as well to first tune the hyperparameters and then test on test data and measure preplexity

*Note*: In language modeling (including RNN-based text generation), the target (y) is just the next word in the sequence.

In [97]:
def split_data_tokens(tokens, seq_len, train_ratio=0.8, val_ratio=0.1):
    total = len(tokens)
    train_end = int(total * train_ratio)
    val_end = int(total * (train_ratio + val_ratio))

    train_tokens = tokens[:train_end]
    val_tokens = tokens[train_end:val_end]
    test_tokens = tokens[val_end:]

    def create_sequences(token_list):
        inputs, targets = [], []
        for i in range(len(token_list) - seq_len):
            inputs.append(token_list[i:i+seq_len])
            targets.append(token_list[i+1:i+seq_len+1])
        return inputs, targets

    X_train, y_train = create_sequences(train_tokens)
    X_val, y_val = create_sequences(val_tokens)
    X_test, y_test = create_sequences(test_tokens)

    return X_train, y_train, X_val, y_val, X_test, y_test


### Parametrs

**embedding_dim:** The size of the dense vector used to represent each word.

Words are encoded into continuous vector space so the model can learn semantic meaning.

It is used in the output size of the nn.Embedding layer.

**hidden_dim:** The size of the hidden state inside the RNN, LSTM, or GRU.
It is used in RNN layer's internal computation and controls how much information the RNN can store at each step.

**optimizer** : The algorithm that updates your model’s weights during training, based on the gradients computed from loss.

It controls how the model learns — how fast, how stable, and whether it converges at all.
It is used on every training step — right after .backward().

**Gradien Clipping**

nn.utils.clip_grad_norm_(model.parameters(), clip)
This is the line doing gradient clipping in our train model function.

Pass information recursively over time steps

Multiply gradients over and over (risking exploding or vanishing gradients)

Are often deeper in the time dimension than feedforward nets

Without clipping, gradients might explode after just a few epochs.

### Vanilla RNN (Bidirectional)

In [30]:
# === TRAINING ===
def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=10, clip=5):
    model.to(device)
    best_val_loss = float('inf')
    best_model_state = None

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output.view(-1, output.shape[-1]), y.view(-1))
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            total_loss += loss.item()
        avg_train_loss = total_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                output = model(X)
                loss = criterion(output.view(-1, output.shape[-1]), y.view(-1))
                val_loss += loss.item()
        avg_val_loss = val_loss / len(val_loader)
        val_perplexity = torch.exp(torch.tensor(avg_val_loss))

        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}, Val Perplexity = {val_perplexity:.2f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()

    return best_val_loss, val_perplexity.item(), best_model_state



In [31]:
from torch.utils.data import TensorDataset

# === MAIN LOOP ===
def run_experiments_on_dataset(name, df, token_col, seq_len=50, batch_size=64):
    print(f"\n📘 Running experiments on dataset: {name}")

    # Flatten token list
    if isinstance(df[token_col].iloc[0], list):
        tokens = list(itertools.chain.from_iterable(df[token_col]))
    else:
        tokens = df[token_col].tolist()

    # Build vocab
    vocab = sorted(set(tokens))
    stoi = {w: i for i, w in enumerate(vocab)}
    indexed_tokens = [stoi[t] for t in tokens]
    vocab_size = len(stoi)

    # Split data
    X_train, y_train, X_val, y_val, _, _ = split_data_tokens(indexed_tokens, seq_len)

    # Datasets & loaders
    train_dataset = TensorDataset(
    torch.tensor(X_train, dtype=torch.long),
    torch.tensor(y_train, dtype=torch.long))

    val_dataset = TensorDataset(
    torch.tensor(X_val, dtype=torch.long),
    torch.tensor(y_val, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Hyperparams
    embedding_dims = [86, 128]
    hidden_dims = [64, 128]
    optimizers = ['adam', 'rmsprop', 'sgd']
    results = []
    best_overall_loss = float('inf')
    best_overall_model = None
    best_config = None

    for embedding_dim in embedding_dims:
        for hidden_dim in hidden_dims:
            for opt_name in optimizers:
                print(f"\n🔧 Config: embed={embedding_dim}, hidden={hidden_dim}, opt={opt_name.upper()}")
                model = VanillaRNNModel_Bi(vocab_size, embedding_dim, hidden_dim, vocab_size)

                if opt_name == 'adam':
                    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
                elif opt_name == 'rmsprop':
                    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
                elif opt_name == 'sgd':
                    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

                criterion = nn.CrossEntropyLoss()
                device = 'cuda' if torch.cuda.is_available() else 'cpu'

                val_loss, val_ppl, model_state = train_model(
                    model, train_loader, val_loader, optimizer, criterion, device
                )

                results.append((embedding_dim, hidden_dim, opt_name, val_loss, val_ppl))

                if val_loss < best_overall_loss:
                    best_overall_loss = val_loss
                    best_overall_model = model_state
                    best_config = (embedding_dim, hidden_dim, opt_name)

    print(f"\n✅ BEST CONFIG FOR {name.upper()}: embed={best_config[0]}, hidden={best_config[1]}, opt={best_config[2].upper()} | Val Loss: {best_overall_loss:.4f}")
    return results, best_config, best_overall_model

In [32]:
#Run it on lemmatized tokens
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from collections import Counter
import numpy as np
import itertools

results_tokens, best_tokens, model_tokens = run_experiments_on_dataset("df_tokens", df_tokens, token_col="Lemma")



📘 Running experiments on dataset: df_tokens

🔧 Config: embed=86, hidden=64, opt=ADAM
Epoch 1: Train Loss = 0.7671, Val Loss = 0.8396, Val Perplexity = 2.32
Epoch 2: Train Loss = 0.0990, Val Loss = 0.8105, Val Perplexity = 2.25
Epoch 3: Train Loss = 0.0905, Val Loss = 0.8121, Val Perplexity = 2.25
Epoch 4: Train Loss = 0.0870, Val Loss = 0.8195, Val Perplexity = 2.27
Epoch 5: Train Loss = 0.0846, Val Loss = 0.8405, Val Perplexity = 2.32
Epoch 6: Train Loss = 0.0826, Val Loss = 0.8610, Val Perplexity = 2.37
Epoch 7: Train Loss = 0.0810, Val Loss = 0.8805, Val Perplexity = 2.41
Epoch 8: Train Loss = 0.0795, Val Loss = 0.8894, Val Perplexity = 2.43
Epoch 9: Train Loss = 0.0782, Val Loss = 0.9134, Val Perplexity = 2.49
Epoch 10: Train Loss = 0.0770, Val Loss = 0.9229, Val Perplexity = 2.52

🔧 Config: embed=86, hidden=64, opt=RMSPROP
Epoch 1: Train Loss = 0.5289, Val Loss = 0.8072, Val Perplexity = 2.24
Epoch 2: Train Loss = 0.0935, Val Loss = 0.7006, Val Perplexity = 2.01
Epoch 3: Train Lo

KeyboardInterrupt: 

The code has an error cause I stopped it to make some chnages and re-run it more efficiently with fewer smart choices.

In [41]:
torch.save(model_tokens,"/content/drive/My Drive/Colab Notebooks/Models/RNN_model_tokens.pth")

**Interpretation of the results**

SGD performed poorly overall. Even the training loss remained high, indicating that the optimizer struggled to converge. As a result, we will not continue using SGD in further experiments.

For ADAM, the training loss dropped rapidly in all cases, but the validation loss began to increase, which is a clear sign of overfitting. However, this issue is not necessarily due to the optimizer itself. It's possible that the model is too complex for the data, leading to high variance.

Higher embedding dimensions consistently resulted in better validation loss. This makes sense because in Shakespeare-style text generation, the language is:

* Rich in stylistic variation

* Heavily dependent on context and syntactic structure

A larger embedding size allows the model to capture these subtle nuances more effectively, leading to improved performance.

--------------

*Next*
We will only use ADAM and RMSprop for other architectures.
For solving the overfitting issue we will add a weight decay to
We will add early stopping to the model cause we see in some cases after 5 epoch there were no change in the loss.

-----------------

So I will choose RMSPROP as the optimizer cbecause in all 3 best models that was the optimizer.

But because of the following fact I will explore two different numbers for both embedding and hidden layers.

* Changing from tokens to POS lemmatization or phrasing changes the
nature of your input representation. The vocabulary distribution, context patterns, and sequence structure may shift. Embedding layers may capture different linguistic patterns, and deeper hidden layers might respond differently to richer syntactic signals.


**Choose the better parameters + early stopping and rerun it for other ways of tokenizing and phrasing**

In [35]:
# Update training model to add early stopping and patience

def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=10, clip=5, early_stopping=False, patience=3):
    model.to(device)
    best_val_loss = float('inf')
    best_model_state = None
    epochs_without_improvement = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output.view(-1, output.shape[-1]), y.view(-1))
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            total_loss += loss.item()
        avg_train_loss = total_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                output = model(X)
                loss = criterion(output.view(-1, output.shape[-1]), y.view(-1))
                val_loss += loss.item()
        avg_val_loss = val_loss / len(val_loader)
        val_perplexity = torch.exp(torch.tensor(avg_val_loss))

        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}, Val Perplexity = {val_perplexity:.2f}")

        # Check for improvement
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        # Early stopping trigger
        if early_stopping and epochs_without_improvement >= patience:
            print(f"⏹️ Early stopping triggered after {epoch+1} epochs.")
            break

    return best_val_loss, val_perplexity.item(), best_model_state


In [36]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import itertools
import numpy as np

def run_experiments_on_dataset(name, df, token_col, seq_len=50, batch_size=64, weight_decay=0.0, patience=3):
    print(f"\n📘 Running experiments on dataset: {name}")

    # Flatten token list
    if isinstance(df[token_col].iloc[0], list):
        tokens = list(itertools.chain.from_iterable(df[token_col]))
    else:
        tokens = df[token_col].tolist()

    # Build vocab
    vocab = sorted(set(tokens))
    stoi = {w: i for i, w in enumerate(vocab)}
    indexed_tokens = [stoi[t] for t in tokens]
    vocab_size = len(stoi)

    # Split data
    X_train, y_train, X_val, y_val, _, _ = split_data_tokens(indexed_tokens, seq_len)

    # Datasets & loaders
    train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.long),
        torch.tensor(y_train, dtype=torch.long))

    val_dataset = TensorDataset(
        torch.tensor(X_val, dtype=torch.long),
        torch.tensor(y_val, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Hyperparams
    embedding_dims = [86, 128]
    hidden_dims = [64, 128]
    results = []
    best_overall_loss = float('inf')
    best_overall_model = None
    best_config = None

    for embedding_dim in embedding_dims:
        for hidden_dim in hidden_dims:
            print(f"\n🔧 Config: embed={embedding_dim}, hidden={hidden_dim}, opt=RMSPROP")
            model = VanillaRNNModel_Bi(vocab_size, embedding_dim, hidden_dim, vocab_size)

            optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=weight_decay)
            criterion = nn.CrossEntropyLoss()
            device = 'cuda' if torch.cuda.is_available() else 'cpu'

            val_loss, val_ppl, model_state = train_model(
                model, train_loader, val_loader, optimizer, criterion, device,
                early_stopping=True, patience=patience
            )

            results.append((embedding_dim, hidden_dim, 'rmsprop', val_loss, val_ppl))

            if val_loss < best_overall_loss:
                best_overall_loss = val_loss
                best_overall_model = model_state
                best_config = (embedding_dim, hidden_dim, 'rmsprop')

    print(f"\n✅ BEST CONFIG FOR {name.upper()}: embed={best_config[0]}, hidden={best_config[1]}, opt=RMSPROP | Val Loss: {best_overall_loss:.4f}, Perplexity: {np.exp(best_overall_loss):.2f}")
    return results, best_config, best_overall_model


In [38]:
#Connect to google
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
#Run it on lemmatized based on part of speech and short and long phrases

import torch

print("Training model on Tokens lemmatized based on Part of Speech")
results_pos_lem, best_pos_lem, model_pos_lem = run_experiments_on_dataset("df_pos_lem", df_pos_lem, token_col="Lemmatized")
torch.save(model_pos_lem, "/content/drive/My Drive/Colab Notebooks/Models/RNN_model_pos_lem.pth")

print("Training model on Short Phrases (Lemmatized)")
results_phrases, best_phrases, model_phrases = run_experiments_on_dataset("df_phrases", df_phrases, token_col="Lemmatized")
torch.save(model_phrases, "/content/drive/My Drive/Colab Notebooks/Models/RNN_model_phrases.pth")

print("Training model on Long Phrases (Lemmatized)")
results_long, best_long, model_long = run_experiments_on_dataset("df_long", df_long, token_col="Lemmatized")
torch.save(model_long, "/content/drive/My Drive/Colab Notebooks/Models/RNN_model_long.pth")



Training model on Tokens lemmatized based on Part of Speech

📘 Running experiments on dataset: df_pos_lem

🔧 Config: embed=86, hidden=64, opt=RMSPROP
Epoch 1: Train Loss = 0.6665, Val Loss = 1.3056, Val Perplexity = 3.69
Epoch 2: Train Loss = 0.0993, Val Loss = 1.1848, Val Perplexity = 3.27
Epoch 3: Train Loss = 0.0961, Val Loss = 1.1468, Val Perplexity = 3.15
Epoch 4: Train Loss = 0.0955, Val Loss = 1.1384, Val Perplexity = 3.12
Epoch 5: Train Loss = 0.0952, Val Loss = 1.1548, Val Perplexity = 3.17
Epoch 6: Train Loss = 0.0949, Val Loss = 1.1839, Val Perplexity = 3.27
Epoch 7: Train Loss = 0.0945, Val Loss = 1.2121, Val Perplexity = 3.36
⏹️ Early stopping triggered after 7 epochs.

🔧 Config: embed=86, hidden=128, opt=RMSPROP
Epoch 1: Train Loss = 0.3387, Val Loss = 1.0238, Val Perplexity = 2.78
Epoch 2: Train Loss = 0.0972, Val Loss = 0.9472, Val Perplexity = 2.58
Epoch 3: Train Loss = 0.0957, Val Loss = 0.9800, Val Perplexity = 2.66
Epoch 4: Train Loss = 0.0945, Val Loss = 1.0137, Va

**Summary of best model performances on training sets:**

(We will use these for test sets from the same input. Bu input I mean the type of tokenizing is different)


✅ BEST CONFIG FOR DF_TOKENS: embed=128, hidden=128, opt=RMSPROP | Val Loss: 0.5770, Val Perplexity = 1.78

✅ BEST CONFIG FOR DF_POS_LEM: embed=86, hidden=128, opt=RMSPROP | Val Loss: 0.9472, Perplexity: 2.58

✅ BEST CONFIG FOR DF_PHRASES: embed=128, hidden=128, opt=RMSPROP | Val Loss: 1.1271, Perplexity: 3.09

✅ BEST CONFIG FOR DF_LONG: embed=86, hidden=128, opt=RMSPROP | Val Loss: 0.9701, Perplexity: 2.64


---------------------
**Takeways**

* We see that modeling on Tokens has the lowest cost. Token-level
inputs are easiest to model for this task. Predicting the next token is relatively straightforward with good performance.

* Perplexity is exponential of the loss:
A perplexity of 1.78 (tokens) means the model is very confident about the next token prediction.

* Lemmatization reduces variation, but POS-aware lemmatization can lead to less natural token sequences (e.g., verbs, nouns, etc., might appear out of typical natural order). So, POS lemmatization is useful for certain NLP tasks (e.g., reducing vocabulary), but here, it introduces a bit more complexity for sequence prediction.

* It was surprising to me that we predicted longer phrases, compared to short phrases with lower perplexity, meaning less confusion.

* This could be the reson behind it:Longer phrases often follow grammatical patterns, such as subject–verb–object or preposition–noun structures. These regularities can actually make predictions more consistent across examples. They also may contain more context and can be more semantically stable.

* So I decided to limit training the other structures only on df_tokens and df_long. This way we will have a token prediction and a phrase prediction.

### LSTM (Bidirectional)

In [45]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import itertools
import numpy as np

def run_experiments_on_dataset(name, df, token_col, seq_len=50, batch_size=64, weight_decay=0.0, patience=3):
    print(f"\n📘 Running experiments on dataset: {name}")

    # Flatten token list
    if isinstance(df[token_col].iloc[0], list):
        tokens = list(itertools.chain.from_iterable(df[token_col]))
    else:
        tokens = df[token_col].tolist()

    # Build vocab
    vocab = sorted(set(tokens))
    stoi = {w: i for i, w in enumerate(vocab)}
    indexed_tokens = [stoi[t] for t in tokens]
    vocab_size = len(stoi)

    # Split data
    X_train, y_train, X_val, y_val, _, _ = split_data_tokens(indexed_tokens, seq_len)

    # Datasets & loaders
    train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.long),
        torch.tensor(y_train, dtype=torch.long))

    val_dataset = TensorDataset(
        torch.tensor(X_val, dtype=torch.long),
        torch.tensor(y_val, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Hyperparams
    embedding_dims = [86, 128]
    hidden_dims = [64, 128]
    results = []
    best_overall_loss = float('inf')
    best_overall_model = None
    best_config = None

    for embedding_dim in embedding_dims:
        for hidden_dim in hidden_dims:
            print(f"\n🔧 Config: embed={embedding_dim}, hidden={hidden_dim}, opt=RMSPROP")
            # We only changed the model
            model = LSTMModel_Bi(vocab_size, embedding_dim, hidden_dim, vocab_size)

            optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=weight_decay)
            criterion = nn.CrossEntropyLoss()
            device = 'cuda' if torch.cuda.is_available() else 'cpu'

            val_loss, val_ppl, model_state = train_model(
                model, train_loader, val_loader, optimizer, criterion, device,
                early_stopping=True, patience=patience
            )

            results.append((embedding_dim, hidden_dim, 'rmsprop', val_loss, val_ppl))

            if val_loss < best_overall_loss:
                best_overall_loss = val_loss
                best_overall_model = model_state
                best_config = (embedding_dim, hidden_dim, 'rmsprop')

    print(f"\n✅ BEST CONFIG FOR {name.upper()}: embed={best_config[0]}, hidden={best_config[1]}, opt=RMSPROP | Val Loss: {best_overall_loss:.4f}, Perplexity: {np.exp(best_overall_loss):.2f}")
    return results, best_config, best_overall_model


In [46]:
#Run it only lemmatized tokens and long phrases to capture both word by word and phrase text prediction

results_tokens, best_tokens, model_tokens = run_experiments_on_dataset("df_tokens", df_tokens, token_col="Lemma")
torch.save(model_tokens, "/content/drive/My Drive/Colab Notebooks/Models/LSTM_model_tokens.pth")

results_long, best_long, model_long = run_experiments_on_dataset("df_long", df_long, token_col="Lemmatized")
torch.save(model_long, "/content/drive/My Drive/Colab Notebooks/Models/LSTM_model_long.pth")




📘 Running experiments on dataset: df_tokens

🔧 Config: embed=86, hidden=64, opt=RMSPROP
Epoch 1: Train Loss = 0.8779, Val Loss = 1.0537, Val Perplexity = 2.87
Epoch 2: Train Loss = 0.0955, Val Loss = 0.9899, Val Perplexity = 2.69
Epoch 3: Train Loss = 0.0903, Val Loss = 1.0288, Val Perplexity = 2.80
Epoch 4: Train Loss = 0.0884, Val Loss = 1.0776, Val Perplexity = 2.94
Epoch 5: Train Loss = 0.0872, Val Loss = 1.1141, Val Perplexity = 3.05
⏹️ Early stopping triggered after 5 epochs.

🔧 Config: embed=86, hidden=128, opt=RMSPROP
Epoch 1: Train Loss = 0.4310, Val Loss = 0.7474, Val Perplexity = 2.11
Epoch 2: Train Loss = 0.0888, Val Loss = 0.7223, Val Perplexity = 2.06
Epoch 3: Train Loss = 0.0862, Val Loss = 0.7510, Val Perplexity = 2.12
Epoch 4: Train Loss = 0.0840, Val Loss = 0.7652, Val Perplexity = 2.15
Epoch 5: Train Loss = 0.0821, Val Loss = 0.7789, Val Perplexity = 2.18
⏹️ Early stopping triggered after 5 epochs.

🔧 Config: embed=128, hidden=64, opt=RMSPROP
Epoch 1: Train Loss = 0

**Comparing RNN and LSTM results**

We know that LSTMs have more parameters due to their gating mechanisms (input, output, forget gates). So even with the same hidden size, an LSTM has more learnable weights than an RNN.

So, it can fit the training data faster (lower train loss). But also has a higher risk of overfitting, especially if the dataset is not large or diverse enough.

But in my results, this is mostly not the case.

✅ RNN: BEST CONFIG FOR DF_TOKENS: embed=128, hidden=128, opt=RMSPROP | Val Loss: 0.5770, Val Perplexity = 1.78

✅ LSTM: BEST CONFIG FOR DF_TOKENS: embed=86, hidden=128, opt=RMSPROP | Val Loss: 0.7223, Perplexity: 2.06

We see that for df_tokens as I checked the best models they have similar train tests (0.08) but RNN had lower validation cost and perplexity so it outperformed LSTM.

**LSTM’s strength is modeling long-range dependencies.**

So  I assumed that on longer phrases it will have a better performance.

✅ RNN: BEST CONFIG FOR DF_LONG: embed=86, hidden=128, opt=RMSPROP | Val Loss: 0.9701, Perplexity: 2.64

✅ LSTM: BEST CONFIG FOR DF_LONG: embed=86, hidden=128, opt=RMSPROP | Val Loss: 1.2620, Perplexity: 3.53

But as wee even with pharses RNN was better then complexity of LSTM might just be unnecessary overhead without payoff.


### GRU (Bidirectional)

In [48]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import itertools
import numpy as np

def run_experiments_on_dataset(name, df, token_col, seq_len=50, batch_size=64, weight_decay=0.0, patience=3):
    print(f"\n📘 Running experiments on dataset: {name}")

    # Flatten token list
    if isinstance(df[token_col].iloc[0], list):
        tokens = list(itertools.chain.from_iterable(df[token_col]))
    else:
        tokens = df[token_col].tolist()

    # Build vocab
    vocab = sorted(set(tokens))
    stoi = {w: i for i, w in enumerate(vocab)}
    indexed_tokens = [stoi[t] for t in tokens]
    vocab_size = len(stoi)

    # Split data
    X_train, y_train, X_val, y_val, _, _ = split_data_tokens(indexed_tokens, seq_len)

    # Datasets & loaders
    train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.long),
        torch.tensor(y_train, dtype=torch.long))

    val_dataset = TensorDataset(
        torch.tensor(X_val, dtype=torch.long),
        torch.tensor(y_val, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Hyperparams
    embedding_dims = [86, 128]
    hidden_dims = [64, 128]
    results = []
    best_overall_loss = float('inf')
    best_overall_model = None
    best_config = None

    for embedding_dim in embedding_dims:
        for hidden_dim in hidden_dims:
            print(f"\n🔧 Config: embed={embedding_dim}, hidden={hidden_dim}, opt=RMSPROP")
            # We only changed the model
            model = GRUModel_Bi(vocab_size, embedding_dim, hidden_dim, vocab_size)

            optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=weight_decay)
            criterion = nn.CrossEntropyLoss()
            device = 'cuda' if torch.cuda.is_available() else 'cpu'

            val_loss, val_ppl, model_state = train_model(
                model, train_loader, val_loader, optimizer, criterion, device,
                early_stopping=True, patience=patience
            )

            results.append((embedding_dim, hidden_dim, 'rmsprop', val_loss, val_ppl))

            if val_loss < best_overall_loss:
                best_overall_loss = val_loss
                best_overall_model = model_state
                best_config = (embedding_dim, hidden_dim, 'rmsprop')

    print(f"\n✅ BEST CONFIG FOR {name.upper()}: embed={best_config[0]}, hidden={best_config[1]}, opt=RMSPROP | Val Loss: {best_overall_loss:.4f}, Perplexity: {np.exp(best_overall_loss):.2f}")
    return results, best_config, best_overall_model


In [51]:
#Run it only lemmatized tokens and long phrases to capture both word by word and phrase text prediction

results_tokens, best_tokens, model_tokens = run_experiments_on_dataset("df_tokens", df_tokens, token_col="Lemma")
torch.save(model_tokens, "/content/drive/My Drive/Colab Notebooks/Models/GRU_model_tokens.pth")

results_long, best_long, model_long = run_experiments_on_dataset("df_long", df_long, token_col="Lemmatized")
torch.save(model_long, "/content/drive/My Drive/Colab Notebooks/Models/GRU_model_long.pth")




📘 Running experiments on dataset: df_tokens

🔧 Config: embed=86, hidden=64, opt=RMSPROP
Epoch 1: Train Loss = 0.7464, Val Loss = 0.9373, Val Perplexity = 2.55
Epoch 2: Train Loss = 0.0936, Val Loss = 0.8210, Val Perplexity = 2.27
Epoch 3: Train Loss = 0.0901, Val Loss = 0.7946, Val Perplexity = 2.21
Epoch 4: Train Loss = 0.0890, Val Loss = 0.7885, Val Perplexity = 2.20
Epoch 5: Train Loss = 0.0883, Val Loss = 0.7901, Val Perplexity = 2.20
Epoch 6: Train Loss = 0.0876, Val Loss = 0.7950, Val Perplexity = 2.21
Epoch 7: Train Loss = 0.0870, Val Loss = 0.8080, Val Perplexity = 2.24
⏹️ Early stopping triggered after 7 epochs.

🔧 Config: embed=86, hidden=128, opt=RMSPROP
Epoch 1: Train Loss = 0.3284, Val Loss = 0.6200, Val Perplexity = 1.86
Epoch 2: Train Loss = 0.0906, Val Loss = 0.6072, Val Perplexity = 1.84
Epoch 3: Train Loss = 0.0886, Val Loss = 0.6326, Val Perplexity = 1.88
Epoch 4: Train Loss = 0.0866, Val Loss = 0.6619, Val Perplexity = 1.94
Epoch 5: Train Loss = 0.0848, Val Loss = 

**Comparing with RNN**

✅ RNN: BEST CONFIG FOR DF_TOKENS: embed=128, hidden=128, opt=RMSPROP | Val Loss: 0.5770, Val Perplexity = 1.78

✅ GRU: BEST CONFIG FOR DF_TOKENS: embed=86, hidden=128, opt=RMSPROP | Val Loss: 0.6072, Perplexity: 1.84

**GRU’s strength is modeling long-range dependencies.**

So  I assumed that on longer phrases it will have a better performance.

✅ RNN: BEST CONFIG FOR DF_LONG: embed=86, hidden=128, opt=RMSPROP | Val Loss: 0.9701, Perplexity: 2.64

✅ GRU: BEST CONFIG FOR DF_LONG: embed=86, hidden=128, opt=RMSPROP | Val Loss: 1.0581, Perplexity: 2.88

Gru performance is still alittle worse than RNN but it is very close so we will some changes to see if we can use its full potential.

------------------

*Changes*:

GRUs can be a bit more sensitive to learning rate so we will go with lr=0.0005

The current seq_len=50 might not give the GRU enough temporal depth to leverage its memory. We will test 100. This increases how much context the model sees in each training sample. GRUs shine when they can track dependencies across longer sequences.

hidden=128 was better in all scenarios. So we son't change that anymore but keep changing the embedding.


In [52]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import itertools
import numpy as np

def run_experiments_on_dataset(name, df, token_col, seq_len=100, batch_size=64, weight_decay=0.0, patience=3):
    print(f"\n📘 Running experiments on dataset: {name}")

    # Flatten token list
    if isinstance(df[token_col].iloc[0], list):
        tokens = list(itertools.chain.from_iterable(df[token_col]))
    else:
        tokens = df[token_col].tolist()

    # Build vocab
    vocab = sorted(set(tokens))
    stoi = {w: i for i, w in enumerate(vocab)}
    indexed_tokens = [stoi[t] for t in tokens]
    vocab_size = len(stoi)

    # Split data
    X_train, y_train, X_val, y_val, _, _ = split_data_tokens(indexed_tokens, seq_len)

    # Datasets & loaders
    train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.long),
        torch.tensor(y_train, dtype=torch.long))

    val_dataset = TensorDataset(
        torch.tensor(X_val, dtype=torch.long),
        torch.tensor(y_val, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Hyperparams
    embedding_dims = [86, 128]
    hidden_dims = [128]
    results = []
    best_overall_loss = float('inf')
    best_overall_model = None
    best_config = None

    for embedding_dim in embedding_dims:
        for hidden_dim in hidden_dims:
            print(f"\n🔧 Config: embed={embedding_dim}, hidden={hidden_dim}, opt=RMSPROP")
            # We only changed the model
            model = GRUModel_Bi(vocab_size, embedding_dim, hidden_dim, vocab_size)

            optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0005, weight_decay=weight_decay)
            criterion = nn.CrossEntropyLoss()
            device = 'cuda' if torch.cuda.is_available() else 'cpu'

            val_loss, val_ppl, model_state = train_model(
                model, train_loader, val_loader, optimizer, criterion, device,
                early_stopping=True, patience=patience
            )

            results.append((embedding_dim, hidden_dim, 'rmsprop', val_loss, val_ppl))

            if val_loss < best_overall_loss:
                best_overall_loss = val_loss
                best_overall_model = model_state
                best_config = (embedding_dim, hidden_dim, 'rmsprop')

    print(f"\n✅ BEST CONFIG FOR {name.upper()}: embed={best_config[0]}, hidden={best_config[1]}, opt=RMSPROP | Val Loss: {best_overall_loss:.4f}, Perplexity: {np.exp(best_overall_loss):.2f}")
    return results, best_config, best_overall_model


In [53]:
#Improvement on GRU: Run it only lemmatized tokens and long phrases to capture both word by word and phrase text prediction

results_tokens, best_tokens, model_tokens = run_experiments_on_dataset("df_tokens", df_tokens, token_col="Lemma")
torch.save(model_tokens, "/content/drive/My Drive/Colab Notebooks/Models/GRU_100_model_tokens.pth")

results_long, best_long, model_long = run_experiments_on_dataset("df_long", df_long, token_col="Lemmatized")
torch.save(model_long, "/content/drive/My Drive/Colab Notebooks/Models/GRU_100_model_long.pth")




📘 Running experiments on dataset: df_tokens

🔧 Config: embed=86, hidden=128, opt=RMSPROP
Epoch 1: Train Loss = 0.4760, Val Loss = 0.6399, Val Perplexity = 1.90
Epoch 2: Train Loss = 0.0454, Val Loss = 0.5902, Val Perplexity = 1.80
Epoch 3: Train Loss = 0.0446, Val Loss = 0.5950, Val Perplexity = 1.81
Epoch 4: Train Loss = 0.0441, Val Loss = 0.6127, Val Perplexity = 1.85
Epoch 5: Train Loss = 0.0436, Val Loss = 0.6229, Val Perplexity = 1.86
⏹️ Early stopping triggered after 5 epochs.

🔧 Config: embed=128, hidden=128, opt=RMSPROP
Epoch 1: Train Loss = 0.4803, Val Loss = 0.6673, Val Perplexity = 1.95
Epoch 2: Train Loss = 0.0448, Val Loss = 0.6164, Val Perplexity = 1.85
Epoch 3: Train Loss = 0.0440, Val Loss = 0.6231, Val Perplexity = 1.86
Epoch 4: Train Loss = 0.0434, Val Loss = 0.6429, Val Perplexity = 1.90
Epoch 5: Train Loss = 0.0428, Val Loss = 0.6544, Val Perplexity = 1.92
⏹️ Early stopping triggered after 5 epochs.

✅ BEST CONFIG FOR DF_TOKENS: embed=86, hidden=128, opt=RMSPROP | 

*Note*:

This is the approach I took so far, I trained a text predictor first and now that I know the models that are performing better than others, I will go to the generation phase and use auto-regreesor and teacher forcing.

First: raining on next-token prediction:

* Simple: Efficient to compute in parallel (whole sequences at once)

* Stable: Loss is clear (cross-entropy with known targets)

* Powerful: Teaches the model grammar, structure, syntax, word flow


Then, We switch to generation mode by:

* Feeding a prompt

* Letting the model generate the next token

* Feeding that token back into the model

* Repeating until you generate the desired length

## Add teacher forcing

Text generation means:

Given a prompt, have the model create something new, one token at a time.

This only happens during inference, after the model is trained.

**So my question was that if Generation is for inference, what does it mean to train it?**

*Answer:* During training, we want the model to learn how to generate, but we don’t actually let it “generate freely” yet.

At each time step, rather than feeding the model its own previous prediction, we feed it the ground truth token from the training data.
Because early in training, the model’s own predictions are garbage.
If we fed those back in, it would learn to follow its own mistakes, and quickly derail.

So instead, we “help it” by saying:

“Hey, here's the correct token. Learn what should come next if you had written this correctly so far.”

That’s teacher forcing.

We Apply teacher forcing as a data preparation trick, not an architecture trick

Uses the teacher forcing ratio to decide whether to:

Use the full input (inputs)

Or slightly shift it (inputs[:, :-1]) to simulate partial input

In [64]:
#This is compatible with both RNN and GRU

def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=10, clip=5,
                early_stopping=False, patience=3, teacher_forcing_ratio=0.5):
    model.to(device)
    best_val_loss = float('inf')
    best_model_state = None
    epochs_without_improvement = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()

            # Apply teacher forcing at batch level
            if random.random() < teacher_forcing_ratio:
                inputs_tf = X
                targets_tf = y
            else:
                inputs_tf = X[:, :-1]
                targets_tf = y[:, :-1]

            outputs = model(inputs_tf)  # full sequence prediction
            loss = criterion(outputs.view(-1, outputs.size(-1)), targets_tf.reshape(-1))

            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # === Validation ===
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                output = model(X)
                loss = criterion(output.view(-1, output.size(-1)), y.view(-1))
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        val_perplexity = torch.exp(torch.tensor(avg_val_loss))

        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}, Val Perplexity = {val_perplexity:.2f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if early_stopping and epochs_without_improvement >= patience:
            print("⏹️ Early stopping triggered.")
            break

    return best_val_loss, val_perplexity.item(), best_model_state


### RNN + teacher forcing  + weight decay (Regularization)

In [65]:

from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import itertools
import numpy as np
import random

def run_experiments_on_dataset(name, df, token_col, seq_len=50, batch_size=64,
                               weight_decay=1e-5, patience=3 , teacher_forcing_ratio=0.5):
    print(f"\n📘 Running experiments on dataset: {name}")

    # Flatten token list
    if isinstance(df[token_col].iloc[0], list):
        tokens = list(itertools.chain.from_iterable(df[token_col]))
    else:
        tokens = df[token_col].tolist()

    # Build vocab
    vocab = sorted(set(tokens))
    stoi = {w: i for i, w in enumerate(vocab)}
    indexed_tokens = [stoi[t] for t in tokens]
    vocab_size = len(stoi)

    # Split data
    X_train, y_train, X_val, y_val, _, _ = split_data_tokens(indexed_tokens, seq_len)

    # Datasets & loaders
    train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.long),
        torch.tensor(y_train, dtype=torch.long))

    val_dataset = TensorDataset(
        torch.tensor(X_val, dtype=torch.long),
        torch.tensor(y_val, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Hyperparams
    embedding_dims = [86, 128]
    hidden_dims = [128]
    results = []
    best_overall_loss = float('inf')
    best_overall_model = None
    best_config = None

    for embedding_dim in embedding_dims:
        for hidden_dim in hidden_dims:
            print(f"\n🔧 Config: embed={embedding_dim}, hidden={hidden_dim}, opt=RMSPROP")
            model = VanillaRNNModel_Bi(vocab_size, embedding_dim, hidden_dim, vocab_size)

            optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=weight_decay)
            criterion = nn.CrossEntropyLoss()
            device = 'cuda' if torch.cuda.is_available() else 'cpu'

            val_loss, val_ppl, model_state = train_model(
                model, train_loader, val_loader, optimizer, criterion, device,
                early_stopping=True, patience=patience
            )

            results.append((embedding_dim, hidden_dim, 'rmsprop', val_loss, val_ppl))

            if val_loss < best_overall_loss:
                best_overall_loss = val_loss
                best_overall_model = model_state
                best_config = (embedding_dim, hidden_dim, 'rmsprop')

    print(f"\n✅ BEST CONFIG FOR {name.upper()}: embed={best_config[0]}, hidden={best_config[1]}, opt=RMSPROP | Val Loss: {best_overall_loss:.4f}, Perplexity: {np.exp(best_overall_loss):.2f}")
    return results, best_config, best_overall_model


In [66]:
#Improvement on RNN: Run it only lemmatized tokens and long phrases to capture both word by word and phrase text prediction

results_tokens, best_tokens, model_tokens = run_experiments_on_dataset("df_tokens", df_tokens, token_col="Lemma")
torch.save(model_tokens, "/content/drive/My Drive/Colab Notebooks/Models/RNN_Teacher_Forcing_model_tokens.pth")

results_long, best_long, model_long = run_experiments_on_dataset("df_long", df_long, token_col="Lemmatized")
torch.save(model_long, "/content/drive/My Drive/Colab Notebooks/Models/RNN_Teacher_Forcing_model_long.pth")




📘 Running experiments on dataset: df_tokens

🔧 Config: embed=86, hidden=128, opt=RMSPROP
Epoch 1: Train Loss = 0.3090, Val Loss = 0.5758, Val Perplexity = 1.78
Epoch 2: Train Loss = 0.1132, Val Loss = 0.5194, Val Perplexity = 1.68
Epoch 3: Train Loss = 0.1106, Val Loss = 0.5067, Val Perplexity = 1.66
Epoch 4: Train Loss = 0.1092, Val Loss = 0.5025, Val Perplexity = 1.65
Epoch 5: Train Loss = 0.1085, Val Loss = 0.5039, Val Perplexity = 1.66
Epoch 6: Train Loss = 0.1082, Val Loss = 0.4985, Val Perplexity = 1.65
Epoch 7: Train Loss = 0.1078, Val Loss = 0.4968, Val Perplexity = 1.64
Epoch 8: Train Loss = 0.1075, Val Loss = 0.5006, Val Perplexity = 1.65
Epoch 9: Train Loss = 0.1074, Val Loss = 0.4995, Val Perplexity = 1.65
Epoch 10: Train Loss = 0.1072, Val Loss = 0.4951, Val Perplexity = 1.64

🔧 Config: embed=128, hidden=128, opt=RMSPROP
Epoch 1: Train Loss = 0.3010, Val Loss = 0.5685, Val Perplexity = 1.77
Epoch 2: Train Loss = 0.1130, Val Loss = 0.5252, Val Perplexity = 1.69
Epoch 3: Tr

**Comparison**

RNN without teacher forcing:

* BEST CONFIG FOR DF_TOKENS: embed=128, hidden=128, opt=RMSPROP | Val Loss: 0.5770, Val Perplexity = 1.78

* BEST CONFIG FOR DF_LONG: embed=86, hidden=128, opt=RMSPROP | Val Loss: 0.9701, Perplexity: 2.64

RNN after teacher forcing:

✅ BEST CONFIG FOR DF_TOKENS: embed=128, hidden=128, opt=RMSPROP | Val Loss: 0.4917, Perplexity: 1.64

✅ BEST CONFIG FOR DF_LONG: embed=128, hidden=128, opt=RMSPROP | Val Loss: 0.7955, Perplexity: 2.22


### GRU + teacher forcing  + weight decay (Regularization)

In [67]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import itertools
import numpy as np
import random

def run_experiments_on_dataset(name, df, token_col, seq_len=100, batch_size=64,
                               weight_decay=1e-5, patience=3, teacher_forcing_ratio=0.5):
    print(f"\n📘 Running experiments on dataset: {name}")

    # Flatten token list
    if isinstance(df[token_col].iloc[0], list):
        tokens = list(itertools.chain.from_iterable(df[token_col]))
    else:
        tokens = df[token_col].tolist()

    # Build vocab
    vocab = sorted(set(tokens))
    stoi = {w: i for i, w in enumerate(vocab)}
    indexed_tokens = [stoi[t] for t in tokens]
    vocab_size = len(stoi)

    # Split data
    X_train, y_train, X_val, y_val, _, _ = split_data_tokens(indexed_tokens, seq_len)

    # Datasets & loaders
    train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.long),
        torch.tensor(y_train, dtype=torch.long))

    val_dataset = TensorDataset(
        torch.tensor(X_val, dtype=torch.long),
        torch.tensor(y_val, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Hyperparams
    embedding_dims = [86, 128]
    hidden_dims = [128]
    results = []
    best_overall_loss = float('inf')
    best_overall_model = None
    best_config = None

    for embedding_dim in embedding_dims:
        for hidden_dim in hidden_dims:
            print(f"\n🔧 Config: embed={embedding_dim}, hidden={hidden_dim}, opt=RMSPROP")
            # We only changed the model
            model = GRUModel_Bi(vocab_size, embedding_dim, hidden_dim, vocab_size)

            optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0005, weight_decay=weight_decay)
            criterion = nn.CrossEntropyLoss()
            device = 'cuda' if torch.cuda.is_available() else 'cpu'

            val_loss, val_ppl, model_state = train_model(
                model, train_loader, val_loader, optimizer, criterion, device,
                early_stopping=True, patience=patience
            )

            results.append((embedding_dim, hidden_dim, 'rmsprop', val_loss, val_ppl))

            if val_loss < best_overall_loss:
                best_overall_loss = val_loss
                best_overall_model = model_state
                best_config = (embedding_dim, hidden_dim, 'rmsprop')

    print(f"\n✅ BEST CONFIG FOR {name.upper()}: embed={best_config[0]}, hidden={best_config[1]}, opt=RMSPROP | Val Loss: {best_overall_loss:.4f}, Perplexity: {np.exp(best_overall_loss):.2f}")
    return results, best_config, best_overall_model


In [68]:
#Improvement on GRU: Run it only lemmatized tokens and long phrases to capture both word by word and phrase text prediction

results_tokens, best_tokens, model_tokens = run_experiments_on_dataset("df_tokens", df_tokens, token_col="Lemma")
torch.save(model_tokens, "/content/drive/My Drive/Colab Notebooks/Models/GRU_Teacher_Forcing_model_tokens.pth")

results_long, best_long, model_long = run_experiments_on_dataset("df_long", df_long, token_col="Lemmatized")
torch.save(model_long, "/content/drive/My Drive/Colab Notebooks/Models/GRU_Teacher_Forcing_model_long.pth")




📘 Running experiments on dataset: df_tokens

🔧 Config: embed=86, hidden=128, opt=RMSPROP
Epoch 1: Train Loss = 0.4936, Val Loss = 0.5289, Val Perplexity = 1.70
Epoch 2: Train Loss = 0.0633, Val Loss = 0.4534, Val Perplexity = 1.57
Epoch 3: Train Loss = 0.0615, Val Loss = 0.4367, Val Perplexity = 1.55
Epoch 4: Train Loss = 0.0608, Val Loss = 0.4363, Val Perplexity = 1.55
Epoch 5: Train Loss = 0.0603, Val Loss = 0.4441, Val Perplexity = 1.56
Epoch 6: Train Loss = 0.0601, Val Loss = 0.4444, Val Perplexity = 1.56
Epoch 7: Train Loss = 0.0599, Val Loss = 0.4371, Val Perplexity = 1.55
⏹️ Early stopping triggered.

🔧 Config: embed=128, hidden=128, opt=RMSPROP
Epoch 1: Train Loss = 0.5185, Val Loss = 0.5350, Val Perplexity = 1.71
Epoch 2: Train Loss = 0.0631, Val Loss = 0.4579, Val Perplexity = 1.58
Epoch 3: Train Loss = 0.0614, Val Loss = 0.4440, Val Perplexity = 1.56
Epoch 4: Train Loss = 0.0607, Val Loss = 0.4406, Val Perplexity = 1.55
Epoch 5: Train Loss = 0.0603, Val Loss = 0.4424, Val P

GRU before teacher forcing:

* BEST CONFIG FOR DF_TOKENS: embed=86, hidden=128, opt=RMSPROP | Val Loss: 0.6072, Perplexity: 1.84

* BEST CONFIG FOR DF_LONG: embed=86, hidden=128, opt=RMSPROP | Val Loss: 1.0581, Perplexity: 2.88

GRU after teacher forcing:

✅ BEST CONFIG FOR DF_TOKENS: embed=86, hidden=128, opt=RMSPROP | Val Loss: 0.4363, Perplexity: 1.55

✅ BEST CONFIG FOR DF_LONG: embed=86, hidden=128, opt=RMSPROP | Val Loss: 0.7485, Perplexity: 2.11


### **Interpretation**

Both RNN and GRU are performing way better with teacher forcing because it provides the model with the correct previous token during training, reducing exposure bias and helping it learn more stable, coherent transitions between words, especially in early training stages when its own predictions are still unreliable.

* Best model for df_tokens: GRU (Val Loss: 0.4363, Perplexity: 1.55)

* Best model for df_Phrases: GRU (Val Loss: 0.7485, Perplexity: 2.11)

We will continue only with best models (Both GRU) to Advanced text generation and testing it on test dataset.


# Evaluation and Analysis

For evaluating text generation, as mentioned in the slides, we have three common metrics: BLEU, ROUGE, and Perplexity.

In our case, Perplexity is the most suitable choice because it does not require a reference text, making it ideal for open-ended text generation tasks like ours.

Perplexity measures how “surprised” the model is by the next word in a sequence. The lower the perplexity, the better the model is at predicting the next word, indicating better fluency, structure, and coherence.

Unlike BLEU and ROUGE, which focus on n-gram overlap with a reference (making them more suitable for tasks like translation or summarization), perplexity evaluates the internal consistency of generated text, which aligns better with the goals of our Shakespeare-style generation task.

WE used Loss (CrossEntropy) and Perplexity for train and validation in the train section.

# Advanced Text Generation

## Temperature-controlled sampling

In text generation, after the model predicts the logits (raw scores) for the next word:

We apply softmax, scaled by a temperature parameter T.

* T < 1.0 → Makes predictions more conservative (sharp softmax, less randomness)

* T = 1.0 → Standard softmax (baseline behavior)

* T > 1.0 → Increases randomness and creativity (flatter softmax)

In [89]:
# Extract the list of tokens
tokens = df_tokens["Lemma"].dropna().tolist()
vocab = sorted(set(tokens))
stoi = {w: i for i, w in enumerate(vocab)}
itos = {i: w for w, i in stoi.items()}
vocab_size = len(vocab)  # should now be 7588
vocab_size

7588

In [90]:
# Create model and load saved state
model = GRUModel_Bi(vocab_size, embedding_dim=86, hidden_dim=128, output_dim=vocab_size)
model.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/Models/GRU_Teacher_Forcing_model_tokens.pth"))
model.eval()

GRUModel_Bi(
  (embedding): Embedding(7588, 86)
  (gru): GRU(86, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=7588, bias=True)
)

In [78]:
def generate_text_GRU(model, seed_text, stoi, itos, seq_len=50, gen_length=50, temperature=1.0, device='cpu'):
    model.to(device)
    model.eval()

    tokens = seed_text.lower().split()
    input_ids = [stoi.get(token, 0) for token in tokens]
    input_tensor = torch.tensor(input_ids[-seq_len:], dtype=torch.long).unsqueeze(0).to(device)

    generated = tokens.copy()

    for _ in range(gen_length):
        with torch.no_grad():
            output = model(input_tensor)
            logits = output[0, -1] / temperature  # Get logits of last token
            probs = F.softmax(logits, dim=-1).cpu().numpy()

        next_token_id = np.random.choice(len(probs), p=probs)
        next_token = itos[next_token_id]
        generated.append(next_token)

        # Update input tensor
        input_ids.append(next_token_id)
        input_tensor = torch.tensor(input_ids[-seq_len:], dtype=torch.long).unsqueeze(0).to(device)

    return " ".join(generated)


In [81]:
#Try different tempratures and generate
import torch.nn.functional as F

for temp in [0.5, 1.0, 1.5]:
    print(f"\n🌡️ Temperature = {temp}")
    print(generate_text_GRU(model, "love is not", stoi, itos, temperature=temp, gen_length=50))



🌡️ Temperature = 0.5
love is not to the if that i do be ##see , and i am thy father , and , my lord , and i am . i am not to hear , it is my lord , and be gone , and the prince with the duke of the king : my

🌡️ Temperature = 1.0
love is not agile . and of the world father camp ' conflict to be ab ##re ' d , sir , she ' s sentenced instinct . patiently abraham , no laurence with her . shall are your own to - cruel ##es , who say ' mean , a ##que hit

🌡️ Temperature = 1.5
love is not liquor panting ##tal besides ##enes celebrated ##lian ; patron ##t thou winter examine blessing ##power apparel once tell perspective weep consequence you of heat post ##gui him intellect ##ssing marrow to sack the holy . - flowing continue their pike most verse ##nate ##ices ##real elizabeth peering these hideous -


The generated text is so wierd cause we used WordPiece or subword tokenizer, which breaks words into parts (e.g., playing → play, ##ing).

And then we lemmatized those pieces, which doesn't make semantic sense for subwords.

So, we will do two things:

* clean the data now to have more meaningful words

* generate the text on the raw tokens and do transfer learning

### Clean the text and revert wordpeice tokenization

In [88]:
def detokenize_wordpiece(tokens):
    cleaned = []
    for token in tokens:
        if token.startswith("##") and cleaned:
            cleaned[-1] += token[2:]
        else:
            cleaned.append(token)
    return " ".join(cleaned).replace(" .", ".").replace(" ,", ",").replace(" '", "'")

In [91]:
import torch.nn.functional as F

for temp in [0.5, 1.0, 1.5]:
    print(f"\n🌡️ Temperature = {temp}")

    raw_generated = generate_text_GRU(
        model,
        seed_text="love is not",
        stoi=stoi,
        itos=itos,
        temperature=temp,
        gen_length=50
    )

    tokens = raw_generated.split()
    cleaned_output = detokenize_wordpiece(tokens)
    print(cleaned_output)



🌡️ Temperature = 0.5
love is not, and the king ? henry bolingbroke : if you besee, and you are now ! i am a for ay, a the prince of york' s eye, and not the king henry' s blood, and he' s

🌡️ Temperature = 1.0
love is not done. friarina : let me, you will notor sold ? i say careful. benvolio : that dare thou pardon me, your cheeked my sword husband her son : then to the turn' s. boy, nor adars !

🌡️ Temperature = 1.5
love is not yet goat recountedssion, fight deter investmbleur while hat malicious gao for that thrill gate bruisebre helengar. him mercury right brittle carefully breedingpas poundots exiled me, preciousvish coffinlining x you born tear toy aplish malice miserable poised


### Retrain the model on raw tokens and then generate

In [82]:
tokens = df_tokens["Token"].dropna().tolist()  # <-- Original full words
vocab = sorted(set(tokens))
stoi = {w: i for i, w in enumerate(vocab)}
itos = {i: w for w, i in stoi.items()}
vocab_size = len(vocab)  # should now be 7588
vocab_size

8471

When we go with raw tokens, vocab size would be differetn so we need to retrain the data for a few epochs

In [83]:
# === Extract cleaned full-word tokens ===
tokens = df_tokens["Token"].dropna().tolist()
vocab = sorted(set(tokens))
stoi = {w: i for i, w in enumerate(vocab)}
itos = {i: w for w, i in stoi.items()}
vocab_size = len(vocab)

# === Encode tokens
indexed_tokens = [stoi[token] for token in tokens]

# === Sequence function
def split_data_tokens(tokens, seq_len=50, train_ratio=0.8, val_ratio=0.1):
    total = len(tokens)
    train_end = int(total * train_ratio)
    val_end = int(total * (train_ratio + val_ratio))

    def create_sequences(data):
        X, y = [], []
        for i in range(len(data) - seq_len):
            X.append(data[i:i+seq_len])
            y.append(data[i+1:i+seq_len+1])
        return X, y

    X_train, y_train = create_sequences(tokens[:train_end])
    X_val, y_val = create_sequences(tokens[train_end:val_end])
    return X_train, y_train, X_val, y_val

X_train, y_train, X_val, y_val = split_data_tokens(indexed_tokens)


In [84]:
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5, clip=5):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output.view(-1, vocab_size), y.view(-1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}: Train Loss = {total_loss / len(train_loader):.4f}")


In [85]:
# Dataloaders
batch_size = 64
train_ds = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
val_ds = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size)

# Model, optimizer
embedding_dim = 86
hidden_dim = 128
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = GRUModel_Bi(vocab_size, embedding_dim, hidden_dim, vocab_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train_model(model, train_loader, val_loader, optimizer, criterion, device)


Epoch 1: Train Loss = 0.6018
Epoch 2: Train Loss = 0.0912
Epoch 3: Train Loss = 0.0850
Epoch 4: Train Loss = 0.0800
Epoch 5: Train Loss = 0.0760


In [86]:
def generate_text_GRU(model, seed_text, stoi, itos, seq_len=50, gen_length=50, temperature=1.0, device='cpu'):
    model.to(device)
    model.eval()
    tokens = seed_text.lower().split()
    input_ids = [stoi.get(token, 0) for token in tokens]
    input_tensor = torch.tensor(input_ids[-seq_len:], dtype=torch.long).unsqueeze(0).to(device)

    generated = tokens.copy()

    for _ in range(gen_length):
        with torch.no_grad():
            output = model(input_tensor)
            logits = output[0, -1] / temperature
            probs = F.softmax(logits, dim=-1).cpu().numpy()
        next_token_id = np.random.choice(len(probs), p=probs)
        next_token = itos[next_token_id]
        generated.append(next_token)
        input_ids.append(next_token_id)
        input_tensor = torch.tensor(input_ids[-seq_len:], dtype=torch.long).unsqueeze(0).to(device)

    return " ".join(generated)


In [87]:
#Try different tempratures and generate
import torch.nn.functional as F

for temp in [0.5, 1.0, 1.5]:
    print(f"\n🌡️ Temperature = {temp}")
    print(generate_text_GRU(model, "love is not", stoi, itos, temperature=temp, gen_length=50))



🌡️ Temperature = 0.5
love is not my true love , and i am less than a king . king edward iv : and tell you , my lord , i will have you ; for i will you and so . henry bo ##ling ##bro ##ke : the lord of buckingham , the king hat ##h

🌡️ Temperature = 1.0
love is not . pol ##ix ##enes : no further than thou has ##t shown the measure by the ears no man . second murderer : and the heaven your people ? him ; remains ; for , thou dos ##t to blank ##s ; and often than he did not here undone

🌡️ Temperature = 1.5
love is not eater nurse above to flight , should knows to see distress as each few eyes , hat ##h made thin from him near my throne . john of offence studied and may strike brook ill abused , if god help thee as the souls shall grant it ##ily . thou


## Analyze stylistic differences between Shakespeare’s original style and the model's output.

There is an observed tradeoff between creativity and coherence.
The higher the temprature.
At lower temprature, the text is smooth and close to original style

* Look at this for example: Temperature = 0.5
love is not my true love , and i am less than a king . king edward iv : and tell you , my lord , i will have you.

Actually this is not bad 😆

It is poetic and understandable.

* At temperature 1: it is still somewhat understandable — but clearly more creative/unfiltered.


* At temperature 1.5: it’s hallucination

**Takeaways form the generated text's style**

✅ Uses Shakespearean character names ("Henry Bolingbroke", "Prince of York")

✅ Has play-like tone, like someone addressing a king

✅ Mimics dialogue form ("Benvolio :", "that dare thou pardon me")

**Improvement**

Some sentances still make not much sense and needs more training on grammatical patterns